In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import plotly.express as px
import warnings; warnings.simplefilter('ignore')

In [ ]:
# Reading in emissions dataframes and then describing basic statistics.

gaiu_emissions = pd.read_csv('../data/gaiu_emissions_to_be_imported.csv')
gaiu_emissions_1990_2014 = pd.read_csv('../data/gaiu_emissions_1990_2014_to_be_imported.csv')
gaiu_emissions.describe(exclude = ['object','int64'])

In [ ]:
gaiu_emissions[gaiu_emissions['Year'].isin([2014])].describe(exclude = ['object','int64'])

In [ ]:
gaiu_emissions[gaiu_emissions['Year'].isin([1990])].describe(exclude = ['object','int64'])

In [ ]:
# Converting relevant variables to integers for ease of display and then showing a dataframe with a carbon dioxide slider.

relevant_variables = ['GDP Per Capita', '% Internet Use', 'CO2 (kts) Emitted', 'CO2 (kts) Emitted Per 1000']
gaiu_emissions_int = gaiu_emissions
for i in relevant_variables: 
    gaiu_emissions_int[i] = gaiu_emissions_int[i].astype(int)

@interact
def show_countries_with_this_many_emissions(column = 'CO2 (kts) Emitted', x = 1000000):
    return gaiu_emissions_int.loc[(gaiu_emissions_int[column] > x) & 
                              (gaiu_emissions["GDP Per Capita"] > np.median(gaiu_emissions["GDP Per Capita"]))]

In [ ]:
# FacetGrid to compare emissions, GDP per capita and % internet use between 1990 and 2014.

graph = sns.FacetGrid(gaiu_emissions_1990_2014, col = "Year", size = 5)

graph.map(sns.kdeplot, 'GDP Per Capita', shade = True, alpha = 0.1, color = "blue")
graph.map(sns.kdeplot, 'CO2 (kts) Emitted', shade = True, alpha = 0.1, color = "red")
graph.map(sns.kdeplot, '% Internet Use', shade = True, alpha = 0.1, color = "green")
plt.legend(['GDP Per Capita', '% Internet Use', 'CO2 (kts) Emitted'], bbox_to_anchor = [1, 1], loc = 'upper left');

In [ ]:
# Scatterplots of each relevant variable over time with a dropdown.

@interact(y = relevant_variables)
def make_scatter(y):
    fig, ax = plt.subplots(figsize=(12, 10))
    sns.scatterplot(x = 'Year', y = y, data = gaiu_emissions, hue = 'Country', palette = 'gist_rainbow')
    plt.legend(bbox_to_anchor = (1.05, 1))
    plt.show()

In [ ]:
# Sunburst graph based on total emissions.

sunburst_data = gaiu_emissions.loc[gaiu_emissions['Year'].isin([2014])]
fig = px.sunburst(sunburst_data, 
                  path = ['Continent', 'Country'], 
                  values = 'CO2 (kts) Emitted', 
                  color = 'GDP Per Capita', 
                  color_continuous_scale = 'earth',
                  color_continuous_midpoint = np.median(sunburst_data['GDP Per Capita']))
fig.show()

In [ ]:
# Sunburst graph based on total emissions per 1,000 people. Very different.

sunburst_data = gaiu_emissions.loc[gaiu_emissions['Year'].isin([2014])]
fig = px.sunburst(sunburst_data, 
                  path = ['Continent', 'Country'], 
                  values = 'CO2 (kts) Emitted Per 1000', 
                  color = 'GDP Per Capita', 
                  color_continuous_scale = 'sunset',
                  color_continuous_midpoint = np.median(sunburst_data['GDP Per Capita']))
fig.show()

In [ ]:
# Simple boxplot comparing 1990 and 2014 emissions per 1,000 people. 

sns.boxplot(x = "Year", y = "CO2 (kts) Emitted Per 1000", data = gaiu_emissions_1990_2014);